In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, ConvLSTM2D, SpatialDropout1D,Bidirectional, Dropout, Conv1D, MaxPooling1D
import keras
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [15]:
df_train = pd.read_csv("E:\\projects\\feature extraction\\Documents\\train1.csv", encoding='cp1252')
df_test = pd.read_csv("E:\\projects\\feature extraction\\Documents\\test1.csv", encoding='cp1252')


In [16]:
print(df_train.head(10))
print(df_test.head(10))

                                                Text Label
0  Issue of U.S.$1,300,000,000 3.389 per cent. Tr...    No
1  Issue Price: 100 per cent. of the Aggregate No...   Yes
2  As set out above, the yield is calculated at t...    No
3  Application has been made by the Issuer (or on...    No
4                         Issue price: 100 per cent.   Yes
5  Issue Price: 100 per cent. of the Aggregate No...   Yes
6               Issue Terms dated 12 September 2017     No
7  Issue of Series 2017-5 USD 400,000,000 Fixed R...    No
8                               Issue Price: 99.425%   Yes
9  Issue Price.................. 99.425% of the a...   Yes
                                                text
0   Issue Price: 102.853 per cent. of the Aggrega...
1  Issue of U.S.$ 250,000,000 Certificates due 20...
2                      Issue Date: 7 September 2016 
3  Issue Price: 99.785 per cent. of the Aggregate...
4  Aggregate Nominal Amount:  Series: U.S.$750,00...
5            Issue of U.S.$750,00

In [17]:
df_train.describe()

,Text,Label
count,18,18
unique,18,2
top,"The Company may, however, from time to time, ...",No
freq,1,10


In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
Text     18 non-null object
Label    18 non-null object
dtypes: object(2)
memory usage: 416.0+ bytes


In [19]:
df_train.shape

(18, 2)

In [20]:
# df_train['text'] = pd.DataFrame(l1)

df_train.head()

,Text,Label
0,"Issue of U.S.$1,300,000,000 3.389 per cent. Tr...",No
1,Issue Price: 100 per cent. of the Aggregate No...,Yes
2,"As set out above, the yield is calculated at t...",No
3,Application has been made by the Issuer (or on...,No
4,Issue price: 100 per cent.,Yes


In [21]:
df_train['Label'] = df_train.Label.eq('Yes').mul(1)

print(df_train)

                                                 Text  Label
0   Issue of U.S.$1,300,000,000 3.389 per cent. Tr...      0
1   Issue Price: 100 per cent. of the Aggregate No...      1
2   As set out above, the yield is calculated at t...      0
3   Application has been made by the Issuer (or on...      0
4                          Issue price: 100 per cent.      1
5   Issue Price: 100 per cent. of the Aggregate No...      1
6                Issue Terms dated 12 September 2017       0
7   Issue of Series 2017-5 USD 400,000,000 Fixed R...      0
8                                Issue Price: 99.425%      1
9   Issue Price.................. 99.425% of the a...      1
10   The Company may, however, from time to time, ...      0
11                       Issuer: JPMorgan Chase & Co.      0
12   Issue Price: 100 per cent. of the Aggregate N...      1
13                            Issue Date: 11 May 2017      0
14  The Issue Price specified above may be more th...      0
15   The aggregate princ

In [22]:
X = df_train['Text']
y = df_train['Label']

print(y)

0     0
1     1
2     0
3     0
4     1
5     1
6     0
7     0
8     1
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
Name: Label, dtype: int32


In [23]:
max_features = 3000
maxlen = 250

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_train.Text.values)

# Pad the data 
X = tokenizer.texts_to_sequences(df_train.Text.values)
X = pad_sequences(X, maxlen=maxlen)

X_final = tokenizer.texts_to_sequences(df_test.text.values)
X_final =  pad_sequences(X_final, maxlen=maxlen)

X_train, X_test, Y_train, Y_test = train_test_split(
    X, 
    df_train.Label, 
    test_size = 0.33, 
    random_state = 42
)
    
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(units = 64, recurrent_dropout=0.2)))
model.add(Dropout(0.5))
model.add(Dense(units = 32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 128)          384000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)               

In [24]:
model.fit(
    X_train, 
    Y_train,
    batch_size = 64,
    epochs = 10,
    verbose = 1,
    validation_data=(X_test, Y_test),
)


Train on 12 samples, validate on 6 samples
Epoch 1/10
12/12 [==============================] - 6s 466ms/step - loss: 0.6889 - accuracy: 0.6667 - val_loss: 0.6936 - val_accuracy: 0.3333
Epoch 2/10
12/12 [==============================] - 1s 123ms/step - loss: 0.6958 - accuracy: 0.4167 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
12/12 [==============================] - 1s 118ms/step - loss: 0.6959 - accuracy: 0.5000 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 4/10
12/12 [==============================] - 1s 119ms/step - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 5/10
12/12 [==============================] - 1s 117ms/step - loss: 0.6898 - accuracy: 0.5833 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 6/10
12/12 [==============================] - 1s 118ms/step - loss: 0.6826 - accuracy: 0.7500 - val_loss: 0.6920 - val_accuracy: 0.5000
Epoch 7/10
12/12 [==============================] - 1s 116ms/step - loss: 0.6877 - accuracy: 0.5833 - va

In [25]:
score, acc = model.evaluate(X_test, Y_test, batch_size = 32)
y_pred_train = model.predict_classes(X_train)
y_pred_test = model.predict_classes(X_test)

print("loss: ", score, "\nacc: ",acc)
print("\nTeste\n", classification_report(Y_test, y_pred_test))
print("\nTreino\n", classification_report(Y_train, y_pred_train))

6/6 [==============================] - 0s 22ms/step
loss:  0.6908762454986572 
acc:  0.5

Teste
               precision    recall  f1-score   support

           0       0.50      1.00      0.67         3
           1       0.00      0.00      0.00         3

    accuracy                           0.50         6
   macro avg       0.25      0.50      0.33         6
weighted avg       0.25      0.50      0.33         6


Treino
               precision    recall  f1-score   support

           0       0.58      1.00      0.74         7
           1       0.00      0.00      0.00         5

    accuracy                           0.58        12
   macro avg       0.29      0.50      0.37        12
weighted avg       0.34      0.58      0.43        12



C:\Users\hp\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#model = model.fit(data_train.text, data_train.target)
y_preds = model.predict_classes(X_final)
print(y_preds)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [32]:
s ="Serial Number:  7277 ISIN:  CA13607GFB55 Principal Amount:  C$1,500,000,000 Issue Date:  August 28, 2019 Maturity Date:  August 28, 2024 Issue Price: 100% Coupon Rate:  2.35%"

In [52]:
s ="Serial Number:  7277 ISIN:  CA13607GFB55 Principal Amount:  C$1,500,000,000 Issue Date:  August 28, 2019 Maturity Date:  August 28, 2024 Issue Price: 100% Coupon Rate:  2.35%"
sub = "issue price"
s = s.lower()
t = s.partition(sub)[2]
print(s.partition(sub)[1])
prilist = t.split(" ")
for i in range(len(prilist)):
    if(not prilist[i].isalnum()):
        print(prilist[i:i+4])
        break

issue price
[':', '100%', 'coupon', 'rate:']
